# Sequential model：一層層順序執行的簡單模型，只有第一層要寫input的規格，其他層的input就是上一層的output。
# Functional API model：可以有多個 input 層或 output 層，結構可以有分叉，適合複雜的模型建立。

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
# 建立Sequential模型
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])
plot_model(model, to_file='/ITHelp/model.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [3]:
# 建立Functional API 模型
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

InputTensor = tf.keras.Input(shape=(100,))
H1 = Dense(10, activation='relu')(InputTensor)
H2 = Dense(20, activation='relu')(H1)
Output = Dense(1, activation='softmax')(H2)

model_API = Model(inputs=InputTensor, outputs=Output)
model_API.summary()

plot_model(model_API, to_file='Functional_API_model.png')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense_4 (Dense)             (None, 10)                1010      
                                                                 
 dense_5 (Dense)             (None, 20)                220       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,251
Trainable params: 1,251
Non-trainable params: 0
_________________________________________________________________
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [5]:
# 有分叉的 Functional API model
# LSTM 3個Input及2個Output
from tensorflow.keras import layers
# Variable-length sequence of ints
title_input = tf.keras.Input(shape=(None,), name="title")  
body_input = tf.keras.Input(shape=(None,), name="body")  
# Binary vectors of size `12`
tags_input = tf.keras.Input(shape=(12,), name="tags")  

# Embed each word in the title into a 64-dimensional vector
title_features = layers.Embedding(10000, 64)(title_input)
body_features = layers.Embedding(10000, 64)(body_input)

# Reduce sequence of embedded words in the body into a single 128/32-dimensional vector
title_features = layers.LSTM(128)(title_features)
body_features = layers.LSTM(32)(body_features)

# Merge all available features into a single large vector via concatenation
x = layers.concatenate([title_features, body_features, tags_input]) # output=128+32+12=172

# Stick a logistic regression for priority prediction/a department classifier on top of the features
priority_pred = layers.Dense(1, name="priority")(x)
department_pred = layers.Dense(4, name="department")(x)


# 有3個Input及2個Output
model_muti = tf.keras.Model(
    inputs=[title_input, body_input, tags_input],
    outputs=[priority_pred, department_pred],
)
model_muti.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 title (InputLayer)             [(None, None)]       0           []                               
                                                                                                  
 body (InputLayer)              [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 64)     640000      ['title[0][0]']                  
                                                                                                  
 embedding_3 (Embedding)        (None, None, 64)     640000      ['body[0][0]']                   
                                                                                            